In [1]:
from tool import *
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.styles import Alignment

# 변수 셋팅

In [2]:
ver = 'Latest'
today = datetime.date.today()
frozen = 5
vendor_list = ['Quanta', 'Pegatron', 'Wingtech']

# 1. GSCP에서 업체별 보정 SP를 엑셀로 만듬

In [3]:
thisweek = get_weekname(today)
frozen_period = [get_weekname_from(thisweek, i) for i in range(frozen+1)] # 확정 구간은 이번주를 포함하여 현재까지 확정되어 있는 주차들을 나타냄. 예를 들어 확정 구간 3주이면 당주 포함 총 4개 주차임
previous_weeklist = get_previous_weeklist(thisweek)
print(f'** {thisweek} 기준의 GSCP SP {ver} 로 작업합니다. 확정구간 : {frozen}주 **')
sp_all = pd.read_excel(get_filename(), sheet_name=0, keep_default_na=False, na_values=['', 'nan']) # GSCP로부터 금주 SP를 데이타프레임화trim_column_name(sp)
trim_column_name(sp_all)

sp_total = pd.DataFrame()
abnormal_sp = pd.DataFrame()

for num, vendor in enumerate(vendor_list):
        print(f'-- 전체 업체 {len(vendor_list)} 개 중 {num+1} 번째, {vendor} 의 데이타 작업 시작합니다.--')
        print(f'    1. {vendor}의 GSCP raw 데이타 {ver} SP를 정리합니다.')
        c1 = sp_all['From Site'].str.contains(vendor.upper())
        c2 = sp_all['Category'].str.contains(ver)
        sp = sp_all[c1 & c2]
        sp = sp[['Mapping Model.Suffix', 'To Site']+get_pattern_from_list(sp.columns, re.compile('-\d\d-'))].reset_index(drop=True)
        sp = sp.convert_dtypes() # 적절한 데이타타입으로 변경
        sp['To Site'] = sp['To Site'].replace(site_name_adjust_map)
        sp = sp.groupby(['Mapping Model.Suffix', 'To Site']).sum()
        index_1 = sp.index # 추후 for 문을 돌릴 때 사용하기 위해 설정함
        
        sp = sp.reset_index()
        sp.insert(2, 'Category', 'SP')
        cat_order = CategoricalDtype(['SP', 'SR', 'PO', 'Real_SP'], ordered=True) # 데이타프레임을 순서대로 표시하기 위해 category 데이타로 설정
        sp['Category'] = sp['Category'].astype(cat_order)
        sp = sp.set_index(['Mapping Model.Suffix', 'To Site', 'Category'])
        
        for i in sp.index:
            for item in ['SR', 'PO', 'Real_SP']:
                sp.loc[i[0], i[1], item ] = 0
        sp = sp.sort_index(level=0)
        
         # 2. 선적 결과 DB에서 이전 주차의 선적값을 SR에 업데이트함
        print(f'    2. {vendor}의 선적 결과 DB 에서 이전 주차의 선적값을 SR에 업데이트합니다.')
        with open(f'D:/Data/{vendor} shipment result DB.bin', 'rb') as f:
            sr = pickle.load(f)
        sr = sr[sr['Week Name'].isin(previous_weeklist)]
        sr = sr.groupby(['Mapping Model.Suffix', 'Ship To', 'Week Name'])['Ship'].sum()
        for x, y, z in sr.index:
            sp.loc[(x, y, 'SR'), z] = sr[(x,y,z)]
            
        # 3. 지난 주 기준의 Open PO 를 통해, 금주 SP의 확정구간 Open PO 값 업데이트
        # 3-1) 확정 구간 중 이번주차의 Open PO 값 업데이트
        print(f'    3. {vendor}의 지난 주 기준 Open PO 를 계산하여 이번주 SP의 확정 구간의 PO값에 업데이트 합니다.')
        opo = get_open_po_at_certainweek(vendor, get_weekname_from(thisweek, -1))
        c1 = (opo['RSD Week Name'] <= thisweek) # PO의 RSD week가 이번주보다 같거나 작은 경우, 즉, 이번주에 해당하는 PO수량만 얻기 위해 필터링 조건임
        opo_th = opo[c1].pivot_table('Open_Qty', index=['Mapping Model.Suffix', 'Ship To'], columns='RSD Week Name', aggfunc=sum).sum(axis=1)

        for x, y in opo_th.index:
            sp.loc[(x, y, 'PO'), thisweek] = opo_th.loc[x, y]

        #  확정 구간 중, 이번 주 이후 구간의 Open PO값 업데이트
        c1 = (opo['RSD Week Name'].isin(frozen_period[1:]))
        opo_fu = opo[c1].pivot_table('Open_Qty', index=['Mapping Model.Suffix', 'Ship To'], columns='RSD Week Name', aggfunc=sum).fillna(0)

        for x, y in opo_fu.index:
            for week in opo_fu.columns:
                sp.loc[(x, y, 'PO'), week] = opo_fu.loc[(x, y), week]
        
#         index_col = ['Type', 'Mapping Model.Suffix', 'To Site', 'Category']
#         df_t = pd.DataFrame(columns=index_col)
#         df_t = df_t.set_index(index_col)
        
        print(f'    4. {vendor}의 조건에 따른 Real SP를 업데이트하고 엑셀 파일로 출력합니다.')
        for x, y in index_1:
            t1 = sp.loc[(x, y, 'SP'), previous_weeklist].sum() - sp.loc[(x, y, 'SR'), previous_weeklist].sum() # 과거 주차들의 SP값과 SR값의 차이를 구한 값
            t2 = sp.loc[(x, y, 'SP'), thisweek] # 이번주차의 SP 값
            t3 = sp.loc[(x, y, 'PO'), thisweek] # 이번주차의 PO 값
            if (t1 == 0): # 과거 주차들의 SP 값과 SR 값의 차이가 없을 때는 실제 SP는 SR값이 되도록 함
                sp.loc[(x, y, 'Real_SP'), previous_weeklist] = sp.loc[(x, y, 'SR'), previous_weeklist]
                sum_sp = sp.loc[(x, y, 'SP'), frozen_period].sum() # 확정구간의 SP의 합계
                sum_po = sp.loc[(x, y, 'PO'), frozen_period].sum() # 확정구간의 PO의 합계
                if sum_sp == sum_po: # 확정 구간 SP와 PO 의 합이 같다면,
                    sp.loc[(x, y, 'Real_SP'), thisweek:] = sp.loc[(x, y, 'SP'), thisweek:] # 실제 SP 는 SP 값으로 반영함
                    reason = 'Normal Case'
                    sp.loc[(x, y), 'Type'] = reason
                else: # 확정 구간 SP와 PO 의 값이 다른 경우라면,
                    sp.loc[(x, y, 'Real_SP'), thisweek:] = sp.loc[(x, y, 'SP'), thisweek:]
                    reason = 'SP-PO is not matched in frozen period'
                    sp.loc[(x, y), 'Type'] = reason
            elif (t1 > 0): # 과거 주차들의 SP값이 SR 값보다 클 경우에는, 
                reason = 'SP is greater than SR'
                sp.loc[(x, y, 'Real_SP')] = sp.loc[(x, y, 'SP')]
                sp.loc[(x, y), 'Type'] = reason
            elif (t1 < 0) & ((t1 + t2 - t3) == 0): # 과거 주차들의 SR 값이 SP 값보다 크고, 그 큰 폭이 이번주차의 SP값에서 이번주차 SP에서 이번주차 PO값을 차감한 값과 동일하다면, 이런경우는 실제 선적했는데 GSCP는 선적을 못한 것으로 carry over한 경우임
                sp.loc[(x, y, 'Real_SP'), previous_weeklist] = sp.loc[(x, y, 'SR'), previous_weeklist] # 과거주차의 실제 SP값은 SR 값으로 함
                sp.loc[(x, y, 'Real_SP'), thisweek] = t3 # 이번주차의 실제 SP값은 이번주차의 PO 값으로 함
                sum_sp = sp.loc[(x, y, 'SP'), frozen_period[1:]].sum()
                sum_po = sp.loc[(x, y, 'PO'), frozen_period[1:]].sum()
                if sum_sp == sum_po:
                    sp.loc[(x, y, 'Real_SP'), frozen_period[1]:] = sp.loc[(x, y, 'SP'), frozen_period[1]:]
                else:
                    sp.loc[(x, y, 'Real_SP'), frozen_period[1]:] = sp.loc[(x, y, 'SP'), frozen_period[1]:]
                    reason = 'SD input delay and SP-PO is not matched in frozen period'
                    sp.loc[(x, y), 'Type'] = reason
                    continue
                reason = 'SD input delay'
                sp.loc[(x, y), 'Type'] = reason
            elif (t1 < 0) & ((t1 + t2 - t3) != 0): # 과거 주차 SR 값이 SP 보다 큰데, 이번주차 SP에서 이번주차 PO값을 차감한 값과 다르다면, SP-PO의 동기화에 뭔가 문제가 있는 것으로 간주함
                reason = 'Something wrong on SP-PO'
#                 sum_sp = sp.loc[(x, y, 'SP'), previous_weeklist+frozen_period].sum()
#                 sum_sr_po = sp.loc[(x, y, 'SR'), previous_weeklist].sum() + sp.loc[(x, y, 'PO'), frozen_period].sum()
                sp.loc[(x, y, 'Real_SP'), previous_weeklist] = sp.loc[(x, y, 'SR'), previous_weeklist]
                sp.loc[(x, y, 'Real_SP'), thisweek:] = sp.loc[(x, y, 'SP'), thisweek:]
                sp.loc[(x, y), 'Type'] = reason

        sp = sp.reset_index()
        sp.insert(0, 'From Site', vendor)
        sp_total = pd.concat([sp_total, sp])

week_num = today.isocalendar()[1] # 이번주의 주차숫자(Week Number)를 구함
update_time = datetime.datetime.now().strftime('%y%m%d%H')
vendor_names = ''
for name in vendor_list:
    vendor_names += '_' + name

sp_total = sp_total.set_index(['From Site', 'Mapping Model.Suffix', 'To Site', 'Category', 'Type'])
filename = f"W{week_num}_GSCP_SP_correction_{vendor_names}_{update_time}_{ver}.xlsx" #이번주 forecast 파일을 저장할 이름 설정
with pd.ExcelWriter("D:/Shipment Plan/GSCP Data/"+filename) as writer:
    sp_total.to_excel(writer, sheet_name='SP Analysis', merge_cells=False)

** 2023-04-24(W17) 기준의 GSCP SP Latest 로 작업합니다. 확정구간 : 5주 **
-- 전체 업체 3 개 중 1 번째, Quanta 의 데이타 작업 시작합니다.--
    1. Quanta의 GSCP raw 데이타 Latest SP를 정리합니다.
    2. Quanta의 선적 결과 DB 에서 이전 주차의 선적값을 SR에 업데이트합니다.
    3. Quanta의 지난 주 기준 Open PO 를 계산하여 이번주 SP의 확정 구간의 PO값에 업데이트 합니다.
    4. Quanta의 조건에 따른 Real SP를 업데이트하고 엑셀 파일로 출력합니다.
-- 전체 업체 3 개 중 2 번째, Pegatron 의 데이타 작업 시작합니다.--
    1. Pegatron의 GSCP raw 데이타 Latest SP를 정리합니다.
    2. Pegatron의 선적 결과 DB 에서 이전 주차의 선적값을 SR에 업데이트합니다.
    3. Pegatron의 지난 주 기준 Open PO 를 계산하여 이번주 SP의 확정 구간의 PO값에 업데이트 합니다.
    4. Pegatron의 조건에 따른 Real SP를 업데이트하고 엑셀 파일로 출력합니다.
-- 전체 업체 3 개 중 3 번째, Wingtech 의 데이타 작업 시작합니다.--
    1. Wingtech의 GSCP raw 데이타 Latest SP를 정리합니다.
    2. Wingtech의 선적 결과 DB 에서 이전 주차의 선적값을 SR에 업데이트합니다.
    3. Wingtech의 지난 주 기준 Open PO 를 계산하여 이번주 SP의 확정 구간의 PO값에 업데이트 합니다.
    4. Wingtech의 조건에 따른 Real SP를 업데이트하고 엑셀 파일로 출력합니다.


In [4]:
fn = get_filename()
wb = load_workbook(fn)

color = PatternFill(fgColor='BFBFBF', fill_type='solid')

# 첫번째 Sheet(SP Analysis)
sh = wb[wb.sheetnames[0]]
for cell in sh[1]:
    cell.fill = color
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
sh.column_dimensions['B'].width = 20
sh.column_dimensions['E'].width = 20
sh.freeze_panes = 'F2'

sh_names = wb.sheetnames
for sh in sh_names:
    wb[sh].sheet_view.showGridLines = False

wb.save(fn)

# 2. 보정된 SP가 있는 Excel File을 불러와 DB에 저장

In [5]:
with open('D:/Data/GSCP raw data.bin', 'rb') as f:
    gscp_df = pickle.load(f)

gscp_df.groupby(['From Site', 'Ver', 'Ref'])['Updated_at'].unique().apply(len).unstack().convert_dtypes()

Ref                    2023-01-02(W01)  2023-01-09(W02)  2023-01-16(W03)  \
From Site Ver                                                              
Pegatron  Final                      1                1                1   
          Latest                     1                1                1   
          ODM Release                1                1                1   
Quanta    Final                      1                1                1   
          Latest                     1                1                1   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-01-23(W04)  2023-01-30(W05)  2023-02-06(W06)  \
From Site Ver                                                              
Pegatron  Final                      1                1                1   
          Latest                     1                1                1   
          ODM Release                1                1                1   
Quanta    Final                      1                1                1   
          Latest                     1                1                1   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-02-13(W07)  2023-02-20(W08)  2023-02-27(W09)  \
From Site Ver                                                              
Pegatron  Final                      1                1                1   
          Latest                     6                5                6   
          ODM Release                1                1                1   
Quanta    Final                      1                1                1   
          Latest                     6                5                6   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-03-06(W10)  2023-03-13(W11)  2023-03-20(W12)  \
From Site Ver                                                              
Pegatron  Final                      1                1                1   
          Latest                     6                2                7   
          ODM Release                1                1                1   
Quanta    Final                      1                1                1   
          Latest                     6                2                7   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-03-27(W13)  2023-04-03(W14)  2023-04-10(W15)  \
From Site Ver                                                              
Pegatron  Final                      1                2                1   
          Latest                     4                4                4   
          ODM Release                1                1                1   
Quanta    Final                      1                2                1   
          Latest                     4                4                4   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-04-17(W16)  2023-04-24(W17)  
From Site Ver                                            
Pegatron  Final                      1                1  
          Latest     

In [6]:
fname = get_filename()
update_time = fname.split('_')[-2].split('.')[0]

sp = pd.read_excel(fname, sheet_name='SP Analysis')
sp = sp[sp['Category'] == 'Real_SP']
sp = sp.drop(columns=['Type', 'Category'])
sp = sp.reset_index(drop=True)
sp.insert(0, 'Ref', get_weekname(today))
sp.insert(1, 'Frozen', frozen)
sp.insert(1, 'Updated_at', update_time)
sp.insert(1, 'Ver', ver)
sp

,Ref,Ver,Updated_at,Frozen,From Site,Mapping Model.Suffix,To Site,2023-02-27(W09),2023-03-06(W10),2023-03-13(W11),...,2023-08-14(W33),2023-08-21(W34),2023-08-28(W35),2023-09-04(W36),2023-09-11(W37),2023-09-18(W38),2023-09-25(W39),2023-10-02(W40),2023-10-09(W41),2023-10-16(W42)
0,2023-04-24(W17),Latest,23042708,5,Quanta,11TC50Q-E.AC10ML,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2023-04-24(W17),Latest,23042708,5,Quanta,11TC50Q-E.AC11U1,ENUS_V,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2023-04-24(W17),Latest,23042708,5,Quanta,11TG50Q-E.AC10KN,EKHQ,6,240,50,...,0,0,0,0,0,0,0,0,0,0
3,2023-04-24(W17),Latest,23042708,5,Quanta,14T90P-G.ARTGK,EKHQ,0,0,49,...,0,0,0,0,0,0,0,0,0,0
4,2023-04-24(W17),Latest,23042708,5,Quanta,14T90Q-G.AA79K,EKHQ,24,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,2023-04-24(W17),Latest,23042708,5,Wingtech,10A30Q-L.AQ14ML,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
184,2023-04-24(W17),Latest,23042708,5,Wingtech,10A30Q-L.AQ24KN,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
185,2023-04-24(W17),Latest,23042708,5,Wingtech,10A30Q-L.AQ28K,EKHQ,0,0,0,...,0,40,0,0,0,0,0,0,0,0
186,2023-04-24(W17),Latest,23042708,5,Wingtech,10A30Q-L.AQ28ML,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
gscp_df = pd.concat([gscp_df, sp])
gscp_df = gscp_df.fillna(0)
gscp_df = gscp_df.convert_dtypes()
gscp_df = gscp_df.reset_index(drop=True)
gscp_df

,Ref,Ver,Updated_at,Frozen,From Site,Mapping Model.Suffix,To Site,2022-08-01(W31),2022-08-08(W32),2022-08-15(W33),...,2022-06-27(W26),2022-07-04(W27),2022-07-11(W28),2022-07-18(W29),2022-07-25(W30),2024-03-18(W12),2024-03-25(W13),2024-04-01(W14),2024-04-08(W15),2024-04-15(W16)
0,2023-02-13(W07),Latest,23021310,2,Quanta,11TG50Q-C.AC10KN,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2023-02-13(W07),Latest,23021310,2,Quanta,11TG50Q-E.AC10KN,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2023-02-13(W07),Latest,23021310,2,Quanta,14T90P-G.ARTGK,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2023-02-13(W07),Latest,23021310,2,Quanta,14T90Q-G.AA75A3,EASL,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2023-02-13(W07),Latest,23021310,2,Quanta,14T90Q-G.AA78B,EEES,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22026,2023-04-24(W17),Latest,23042708,5,Wingtech,10A30Q-L.AQ14ML,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22027,2023-04-24(W17),Latest,23042708,5,Wingtech,10A30Q-L.AQ24KN,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22028,2023-04-24(W17),Latest,23042708,5,Wingtech,10A30Q-L.AQ28K,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22029,2023-04-24(W17),Latest,23042708,5,Wingtech,10A30Q-L.AQ28ML,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 6. GSCP 데이타에 매주차별로 업체별 data가 있는 지 확인용

In [8]:
gscp_df.groupby(['From Site', 'Ver', 'Ref'])['Updated_at'].unique().apply(len).unstack().convert_dtypes()

Ref                    2023-01-02(W01)  2023-01-09(W02)  2023-01-16(W03)  \
From Site Ver                                                              
Pegatron  Final                      1                1                1   
          Latest                     1                1                1   
          ODM Release                1                1                1   
Quanta    Final                      1                1                1   
          Latest                     1                1                1   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-01-23(W04)  2023-01-30(W05)  2023-02-06(W06)  \
From Site Ver                                                              
Pegatron  Final                      1                1                1   
          Latest                     1                1                1   
          ODM Release                1                1                1   
Quanta    Final                      1                1                1   
          Latest                     1                1                1   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-02-13(W07)  2023-02-20(W08)  2023-02-27(W09)  \
From Site Ver                                                              
Pegatron  Final                      1                1                1   
          Latest                     6                5                6   
          ODM Release                1                1                1   
Quanta    Final                      1                1                1   
          Latest                     6                5                6   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-03-06(W10)  2023-03-13(W11)  2023-03-20(W12)  \
From Site Ver                                                              
Pegatron  Final                      1                1                1   
          Latest                     6                2                7   
          ODM Release                1                1                1   
Quanta    Final                      1                1                1   
          Latest                     6                2                7   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-03-27(W13)  2023-04-03(W14)  2023-04-10(W15)  \
From Site Ver                                                              
Pegatron  Final                      1                2                1   
          Latest                     4                4                4   
          ODM Release                1                1                1   
Quanta    Final                      1                2                1   
          Latest                     4                4                4   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-04-17(W16)  2023-04-24(W17)  
From Site Ver                                            
Pegatron  Final                      1                1  
          Latest     

# 7. 최종 업데이트된 데이터 저장

In [9]:
with open('D:Data/GSCP raw data.bin', 'wb') as f:
    pickle.dump(gscp_df, f)

In [10]:
# DB 백업용 폴더에도 저장
with open('D:/Data/DB backup/GSCP raw data.bin', 'wb') as f:
    pickle.dump(gscp_df, f)

## ODM 으로 발송한 SP를 GSCP DB에 업데이트

### 1) Quanta, Pegatron의 물동일 경우

In [3]:
frozen = 5 # 확정구간 설정

In [4]:
# 23년 1월 이후에는 아래로 사용
fname = get_filename()
tab_name = f"W{fname.split('_')[-1].split('.')[0][:-1]}"
thisweek = get_weekname(datetime.date(2023, int(tab_name[1:3]), int(tab_name[3:5])))
ver = 'ODM Release'
updated_time = (datetime.date(2023, int(tab_name[1:3]), int(tab_name[3:5])) + datetime.timedelta(2)).strftime('%y%m%d') + '00'

vendor = fname.split('_')[2].split(' ')[0]

sp = get_PC_ODM_shipment_plan(fname, vendor.upper(), tab_name).groupby(['Mapping Model.Suffix', 'To Site']).sum(numeric_only=True)
sp = sp.loc[:, get_previous_weeklist(thisweek)[0]:] # Shipment plan에 이전달의 주차 가 없을 때는 본 명령을 주석 처리해야 함
sp['Sum'] = sp.sum(axis=1)
sp = sp[sp['Sum'] > 0]
sp = sp.drop(columns='Sum')
sp = sp.reset_index()
sp.insert(0, 'Ref', thisweek)
sp.insert(1, 'Frozen', frozen)
sp.insert(1, 'Updated_at', updated_time)
sp.insert(1, 'Ver', ver)
sp.insert(2, 'From Site', vendor)
sp

,Ref,Ver,From Site,Updated_at,Frozen,Mapping Model.Suffix,To Site,2023-02-27(W09),2023-03-06(W10),2023-03-13(W11),...,2023-10-23(W43),2023-10-30(W44),2023-11-06(W45),2023-11-13(W46),2023-11-20(W47),2023-11-27(W48),2023-12-04(W49),2023-12-11(W50),2023-12-18(W51),2023-12-25(W52)
0,2023-04-24(W17),ODM Release,Pegatron,23042600,5,13U70Q-G.AR31J,EJJP,23,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2023-04-24(W17),ODM Release,Pegatron,23042600,5,14T30Q-E.A710ML,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2023-04-24(W17),ODM Release,Pegatron,23042600,5,14TN30Q-E.AO14KN,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2023-04-24(W17),ODM Release,Pegatron,23042600,5,14TW30Q-E.AO14ML,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2023-04-24(W17),ODM Release,Pegatron,23042600,5,14U30P-E.A716K,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2023-04-24(W17),ODM Release,Pegatron,23042600,5,14U30P-E.A726K,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2023-04-24(W17),ODM Release,Pegatron,23042600,5,17U70Q-G.ARLGL,EKHQ,468,1860,0,...,0,0,0,0,0,0,0,0,0,0
7,2023-04-24(W17),ODM Release,Pegatron,23042600,5,17U70Q-G.AS7TL,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2023-04-24(W17),ODM Release,Pegatron,23042600,5,17U70Q-P.AD78A8,ENCI,0,0,15,...,0,0,0,0,0,0,0,0,0,0
9,2023-04-24(W17),ODM Release,Pegatron,23042600,5,17U70Q-P.AP5GL,EKHQ,10,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
with open('D:/Data/GSCP raw data.bin', 'rb') as f:
    gscp_df = pickle.load(f)

gscp_df.groupby(['From Site', 'Ver', 'Ref'])['Updated_at'].unique().apply(len).unstack().convert_dtypes()

Ref                    2023-01-02(W01)  2023-01-09(W02)  2023-01-16(W03)  \
From Site Ver                                                              
Pegatron  Final                      1                1                1   
          Latest                     1                1                1   
          ODM Release                1                1                1   
Quanta    Final                      1                1                1   
          Latest                     1                1                1   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-01-23(W04)  2023-01-30(W05)  2023-02-06(W06)  \
From Site Ver                                                              
Pegatron  Final                      1                1                1   
          Latest                     1                1                1   
          ODM Release                1                1                1   
Quanta    Final                      1                1                1   
          Latest                     1                1                1   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-02-13(W07)  2023-02-20(W08)  2023-02-27(W09)  \
From Site Ver                                                              
Pegatron  Final                      1                1                1   
          Latest                     6                5                6   
          ODM Release                1                1                1   
Quanta    Final                      1                1                1   
          Latest                     6                5                6   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-03-06(W10)  2023-03-13(W11)  2023-03-20(W12)  \
From Site Ver                                                              
Pegatron  Final                      1                1                1   
          Latest                     6                2                7   
          ODM Release                1                1                1   
Quanta    Final                      1                1                1   
          Latest                     6                2                7   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-03-27(W13)  2023-04-03(W14)  2023-04-10(W15)  \
From Site Ver                                                              
Pegatron  Final                      1                2                1   
          Latest                     4                4                4   
          ODM Release                1                1                1   
Quanta    Final                      1                2                1   
          Latest                     4                4                4   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-04-17(W16)  2023-04-24(W17)  
From Site Ver                                            
Pegatron  Final                      1                1  
          Latest     

In [6]:
gscp_df = pd.concat([gscp_df, sp])
gscp_df = gscp_df.fillna(0)
gscp_df = gscp_df.convert_dtypes()
gscp_df = gscp_df.reset_index(drop=True)
gscp_df

,Ref,Ver,Updated_at,Frozen,From Site,Mapping Model.Suffix,To Site,2022-08-01(W31),2022-08-08(W32),2022-08-15(W33),...,2022-06-27(W26),2022-07-04(W27),2022-07-11(W28),2022-07-18(W29),2022-07-25(W30),2024-03-18(W12),2024-03-25(W13),2024-04-01(W14),2024-04-08(W15),2024-04-15(W16)
0,2023-02-13(W07),Latest,23021310,2,Quanta,11TG50Q-C.AC10KN,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2023-02-13(W07),Latest,23021310,2,Quanta,11TG50Q-E.AC10KN,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2023-02-13(W07),Latest,23021310,2,Quanta,14T90P-G.ARTGK,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2023-02-13(W07),Latest,23021310,2,Quanta,14T90Q-G.AA75A3,EASL,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2023-02-13(W07),Latest,23021310,2,Quanta,14T90Q-G.AA78B,EEES,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22199,2023-04-24(W17),ODM Release,23042600,5,Pegatron,17U70Q-P.AP5GL,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22200,2023-04-24(W17),ODM Release,23042600,5,Pegatron,17U70Q-P.AP7GL,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22201,2023-04-24(W17),ODM Release,23042600,5,Pegatron,17U70Q-P.AS7CL,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22202,2023-04-24(W17),ODM Release,23042600,5,Pegatron,17UD70Q-P.AX70K,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 6. GSCP 데이타에 매주차별로 업체별 data가 있는 지 확인용

In [7]:
gscp_df.groupby(['From Site', 'Ver', 'Ref'])['Updated_at'].unique().apply(len).unstack().convert_dtypes()

Ref                    2023-01-02(W01)  2023-01-09(W02)  2023-01-16(W03)  \
From Site Ver                                                              
Pegatron  Final                      1                1                1   
          Latest                     1                1                1   
          ODM Release                1                1                1   
Quanta    Final                      1                1                1   
          Latest                     1                1                1   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-01-23(W04)  2023-01-30(W05)  2023-02-06(W06)  \
From Site Ver                                                              
Pegatron  Final                      1                1                1   
          Latest                     1                1                1   
          ODM Release                1                1                1   
Quanta    Final                      1                1                1   
          Latest                     1                1                1   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-02-13(W07)  2023-02-20(W08)  2023-02-27(W09)  \
From Site Ver                                                              
Pegatron  Final                      1                1                1   
          Latest                     6                5                6   
          ODM Release                1                1                1   
Quanta    Final                      1                1                1   
          Latest                     6                5                6   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-03-06(W10)  2023-03-13(W11)  2023-03-20(W12)  \
From Site Ver                                                              
Pegatron  Final                      1                1                1   
          Latest                     6                2                7   
          ODM Release                1                1                1   
Quanta    Final                      1                1                1   
          Latest                     6                2                7   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-03-27(W13)  2023-04-03(W14)  2023-04-10(W15)  \
From Site Ver                                                              
Pegatron  Final                      1                2                1   
          Latest                     4                4                4   
          ODM Release                1                1                1   
Quanta    Final                      1                2                1   
          Latest                     4                4                4   
          ODM Release                1                1                1   
Wingtech  Final                   <NA>             <NA>             <NA>   
          Latest                  <NA>             <NA>             <NA>   

Ref                    2023-04-17(W16)  2023-04-24(W17)  
From Site Ver                                            
Pegatron  Final                      1                1  
          Latest     

# 7. 최종 업데이트된 데이타 저장

In [8]:
with open('D:/Data/GSCP raw data.bin', 'wb') as f:
    pickle.dump(gscp_df, f)

In [9]:
# DB 백업용 폴더에도 저장
with open('D:/Data/DB backup/GSCP raw data.bin', 'wb') as f:
    pickle.dump(gscp_df, f)

### 2) Wanlida 의 물동일 경우

In [ ]:
fname = get_filename()
vendor = fname.split('/')[1]
thisweek = get_weekname(datetime.date.fromisoformat(f"{fname.split('_')[2].split('/')[1][:4]}-{fname.split('_')[2].split('/')[1][4:6]}-{fname.split('_')[2].split('/')[1][6:]}"))
updated_time = f"{fname.split('_')[2].split('/')[1][2:]}00"
ver = 'ODM Release'
ref = thisweek
frozen = 3

sp = pd.read_excel(fname, sheet_name=thisweek, skiprows=1).groupby(['Mapping Model.Suffix', 'To Site']).sum()
sp['Sum'] = sp.sum(axis=1)
sp = sp[sp['Sum'] > 0]
sp = sp.drop(columns='Sum')
sp = sp.reset_index()
sp.insert(0, 'Ref', thisweek)
sp.insert(1, 'Frozen', frozen)
sp.insert(1, 'Updated_at', updated_time)
sp.insert(1, 'Ver', ver)
sp.insert(2, 'From Site', vendor)
sp

In [ ]:
df = pd.read_excel('D:/Shipment Plan/GSCP Data/W5_GSCP_SP_correction__Quanta_Pegatron_23020109_Latest.xlsx', sheet_name='SP Analysis')
df

In [ ]:
df['Series'] = df['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model)
c1 = df['From Site'] == 'Quanta'
df[c1]

In [ ]:
c2 = df['Category'].isin(['Real_SP', 'PO'])

In [ ]:
df[c1& c2].pivot_table(get_weeklist(df), index=['Series', 'Category'], aggfunc=sum).to_clipboard()

In [12]:
get_sp_po_gap('Quanta')

2023-03-20(W12)         2023-03-27(W13)         2023-04-03(W14)  \
Category              PO Real_SP              PO Real_SP              PO   
Series                                                                     
11T50Q               140     140               0       0               0   
14T90P                 0       0               0       0               0   
14T90Q                20      20              15      15               0   
14T90R              1341    1341               0       0             210   
15U40Q                50      50               0       0               0   
15U40R               884     279             260     260            1480   
15U50P                50      50             140     140              80   
15U50Q              2177    2177            1550    1500            1380   
15U50R                 0       0             100     100             190   
16T90Q               635     605             420     420               0   
16T90R               933     933             290     290              35   
SUM                 6230    5595            2775    2725            3375   

                 2023-04-10(W15)          
Category Real_SP              PO Real_SP  
Series                                    
11T50Q         0               0       0  
14T90P         0               0       0  
14T90Q         0               0       0  
14T90R       300               0       0  
15U40Q         0               0     900  
15U40R      1480               0     160  
15U50P        80               0      30  
15U50Q      1380               0     700  
15U50R       190               0     430  
16T90Q        15               0     215  
16T90R       120               0      90  
SUM         3565               0    2525